In [22]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel
import scipy.stats as stats
import sys

sys.path.append("../")
import vuong_tests5

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
class OLS_loglike(GenericLikelihoodModel):
    
    def __init__(self, *args,ols=False, **kwargs):
        super(OLS_loglike,self).__init__(*args,**kwargs)
        self.ols = ols

    def loglikeobs(self, params):
        y = self.endog
        x = self.exog
        mu_y = np.matmul(x,params)  
        resid = y - mu_y
        sigma = np.sqrt(np.sum(resid**2)/resid.shape[0])
        pr_y = stats.norm.logpdf( resid, loc=0,scale=sigma )
        return pr_y

In [24]:
def setup_model(yn,xn):
    """setup models for ease"""
    model1 = sm.OLS(yn,sm.add_constant(xn))
    model1_fit = model1.fit(disp=False)
    params1 = (model1_fit.params)
    model1_deriv = OLS_loglike(yn,sm.add_constant(xn))
    ll1 = model1_deriv.loglikeobs(model1_fit.params)
    grad1 =  model1_deriv.score_obs(model1_fit.params)    
    hess1 = model1_deriv.hessian(model1_fit.params)
    return ll1,grad1,hess1,params1


def setup_test(yn,xn):
    lls = []
    grads = []
    hesss = []
    params = []
    for i in range(xn.shape[1]):
        ll,grad,hess,param = setup_model(yn,xn[:,i])
        lls.append(ll)
        grads.append(grad)
        hesss.append(hess)
        params.append(param)
    
    #determine 1 and 2
    ind1 = 0
    ind2 = 1
    return (lls[ind1],grads[ind1],hesss[ind1],params[ind1],
            lls[ind2],grads[ind2],hesss[ind2],params[ind2])

In [25]:
def gen_data(nobs=1000, a=0.25):
    num_params=2
    x = np.random.normal(scale=1., size=(nobs,num_params))
    e = np.random.pareto(1.5, size=nobs)
    y = 1 + a*x.sum(axis=1) + e
    return y,x,nobs

num_sims=100

In [26]:
gen_data_ex = lambda : gen_data(nobs=100, a=0.0)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_test)
vuong_tests5.print_mc(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 1.00 & 1.00 & 0.99 & 0.99 & 0.97 & 1.00   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.01 & 0.00   \\
Model 2 & 0.00 & 0.00 & 0.01 & 0.01 & 0.02 & 0.00   \\
\hline
\end{tabular}


In [27]:
gen_data_ex = lambda : gen_data(nobs=100, a=0.1)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_test)
vuong_tests5.print_mc(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.98 & 1.00 & 0.96 & 0.96 & 0.96 & 0.99   \\
Model 1 & 0.00 & 0.00 & 0.03 & 0.03 & 0.03 & 0.01   \\
Model 2 & 0.02 & 0.00 & 0.01 & 0.01 & 0.01 & 0.00   \\
\hline
\end{tabular}


In [28]:
gen_data_ex = lambda : gen_data(nobs=100, a=1.0)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_test)
vuong_tests5.print_mc(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.87 & 0.97 & 0.87 & 0.87 & 0.89 & 0.96   \\
Model 1 & 0.04 & 0.00 & 0.04 & 0.04 & 0.04 & 0.02   \\
Model 2 & 0.09 & 0.03 & 0.09 & 0.09 & 0.07 & 0.02   \\
\hline
\end{tabular}


In [32]:
gen_data_ex = lambda : gen_data(nobs=250, a=1.0)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_test)
vuong_tests5.print_mc(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.92 & 0.98 & 0.89 & 0.89 & 0.92 & 1.00   \\
Model 1 & 0.04 & 0.01 & 0.06 & 0.07 & 0.04 & 0.00   \\
Model 2 & 0.04 & 0.01 & 0.05 & 0.04 & 0.04 & 0.00   \\
\hline
\end{tabular}


In [33]:
gen_data_ex = lambda : gen_data(nobs=500, a=1.0)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_test)
vuong_tests5.print_mc(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.85 & 0.91 & 0.87 & 0.86 & 0.86 & 0.99   \\
Model 1 & 0.09 & 0.04 & 0.08 & 0.09 & 0.08 & 0.00   \\
Model 2 & 0.06 & 0.05 & 0.05 & 0.05 & 0.06 & 0.01   \\
\hline
\end{tabular}


In [29]:
gen_data_ex = lambda : gen_data(nobs=100, a=2.0)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_test)
vuong_tests5.print_mc(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.87 & 0.94 & 0.81 & 0.82 & 0.83 & 0.99   \\
Model 1 & 0.08 & 0.03 & 0.10 & 0.10 & 0.09 & 0.01   \\
Model 2 & 0.05 & 0.03 & 0.09 & 0.08 & 0.08 & 0.00   \\
\hline
\end{tabular}
